In [18]:
from langchain_community.tools import DuckDuckGoSearchRun
import yfinance
import json

def get_ticker(inputs: dict):
    ddgs = DuckDuckGoSearchRun()
    print(f"\n[get_ticker] {inputs}")
    company_name: str = inputs["company_name"]
    return ddgs.run(f"Ticker symbol for {company_name}")

def get_income_statement(inputs: dict):
    print(f"\n[get_income_statement] {inputs}")
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs: dict):
    print(f"\n[get_balance_sheet] {inputs}")
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs: dict):
    print(f"\n[get_daily_stock_performance] {inputs}")
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
                "additionalProperties": False
            },
            "strict": True
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
                "additionalProperties": False
            },
            "strict": True
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
                "additionalProperties": False
            },
            "strict": True
        },
    },
]

In [27]:
import openai as client

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4o-mini",
    temperature=0,
    tools=functions,
)
print(assistant)

Assistant(id='asst_mDpaG2aEVn7yacQmzaZewQ68', created_at=1735615848, description=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', metadata={}, model='gpt-4o-mini', name='Investor Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker'], 'additionalProperties': False}, strict=True), type='function'), Func

In [44]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Palantir stock is a good investment."
        }
    ]
)
print(thread)

Thread(id='thread_unAj3Gzc1EbjG7UOqpOWIe2F', created_at=1735625089, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [45]:
from typing import Iterable
from openai import AssistantEventHandler
from openai.types.beta.threads import Run, run_submit_tool_outputs_params
from rich import print_json
from typing_extensions import override

class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        if event.event == 'thread.run.requires_action':
            print(f"\n[requires_action] {', '.join(tool.function.name for tool in event.data.required_action.submit_tool_outputs.tool_calls)}", end="\n", flush=True)
            self.handle_requires_action(event.data)

    def handle_requires_action(self, data: Run):
        tool_outputs: Iterable[run_submit_tool_outputs_params.ToolOutput] = []
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            func = None
            if tool.function.name == "get_ticker":
                func = get_ticker
            elif tool.function.name == "get_income_statement":
                func = get_income_statement
            elif tool.function.name == "get_balance_sheet":
                func = get_balance_sheet
            elif tool.function.name == "get_daily_stock_performance":
                func = get_daily_stock_performance
            else:
                raise ValueError(f"Unknown tool: {tool.function.name}")

            result = func(json.loads(tool.function.arguments))
            print(f"\n[Result ({tool.function.name})]", end="\n", flush=True)
            try:
                print_json(result)
            except:
                print(result)
            print()

            tool_outputs.append({"tool_call_id": tool.id, "output": result})

        
        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs)

    def submit_tool_outputs(self, tool_outputs: Iterable[run_submit_tool_outputs_params.ToolOutput]):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


[requires_action] get_ticker

[get_ticker] {'company_name': 'Palantir Technologies'}


/Users/ryan/Documents/github/JWWon/fullstack-gpt/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



[Result (get_ticker)]
Get a real-time Palantir Technologies Inc. (PLTR) stock price quote with breaking news, financials, statistics, charts and more. ... Ticker Symbol PLTR. Website https://www.palantir.com. Full Company Profile. Financial Performance. Financial Statements. Analyst Forecast. Palantir Technologies Inc. (PLTR.NASDAQ): Stock quote, stock chart, quotes, analysis, advice, financials and news for Stock Palantir Technologies Inc. | Nasdaq: PLTR | Nasdaq. ... Palantir Technologies Inc. is engaged in building software to assist in counterterrorism investigations and operations. It has built four principal software platforms ... The latest Palantir stock prices, stock quotes, news, and PLTR history to help you invest and trade smarter. ... (year end quote, basic EPS) ... Palantir Technologies, Inc. engages in the business ... Palantir Technologies Inc. builds and deploys software platforms for the intelligence community to assist in counterterrorism investigations and operatio

"{\"1703980800000\":{\"Tax Effect Of Unusual Items\":0.0,\"Tax Rate For Calcs\":0.083158,\"Normalized EBITDA\":273915000.0,\"Total Unusual Items\":null,\"Total Unusual Items Excluding Goodwill\":null,\"Net Income From Continuing Operation Net Minority Interest\":209825000.0,\"Reconciled Depreciation\":33354000.0,\"Reconciled Cost Of Revenue\":431105000.0,\"EBITDA\":273915000.0,\"EBIT\":240561000.0,\"Net Interest Income\":129102000.0,\"Interest Expense\":3470000.0,\"Interest Income\":132572000.0,\"Normalized Income\":209825000.0,\"Net Income From Continuing And Discontinued Operation\":209825000.0,\"Total Expenses\":2105046000.0,\"Total Operating Income As Reported\":119966000.0,\"Diluted Average Shares\":2297927000.0,\"Basic Average Shares\":2147446000.0,\"Diluted EPS\":0.09,\"Basic EPS\":0.1,\"Diluted NI Availto Com Stockholders\":209825000.0,\"Average Dilution Earnings\":null,\"Net Income Common Stockholders\":209825000.0,\"Otherunder Preferred Stock Dividend\":null,\"Net Income\":209825000.0,\"Minority Interests\":-7550000.0,\"Net Income Including Noncontrolling Interests\":217375000.0,\"Net Income Continuous Operations\":217375000.0,\"Tax Provision\":19716000.0,\"Pretax Income\":237091000.0,\"Other Income Expense\":-11977000.0,\"Other Non Operating Income Expenses\":-11977000.0,\"Gain On Sale Of Security\":null,\"Net Non Operating Interest Income Expense\":129102000.0,\"Interest Expense Non Operating\":3470000.0,\"Interest Income Non Operating\":132572000.0,\"Operating Income\":119966000.0,\"Operating Expense\":1673941000.0,\"Research And Development\":404624000.0,\"Selling General And Administration\":1269317000.0,\"Selling And Marketing Expense\":744992000.0,\"General And Administrative Expense\":524325000.0,\"Other Gand A\":524325000.0,\"Gross Profit\":1793907000.0,\"Cost Of Revenue\":431105000.0,\"Total Revenue\":2225012000.0,\"Operating Revenue\":2225012000.0},\"1672444800000\":{\"Tax Effect Of Unusual Items\":0.0,\"Tax Rate For Calcs\":0.21,\"Normalized EBITDA\":-334447000.0,\"Total Unusual Items\":null,\"Total Unusual Items Excluding Goodwill\":null,\"Net Income From Continuing Operation Net Minority Interest\":-373705000.0,\"Reconciled Depreciation\":22522000.0,\"Reconciled Cost Of Revenue\":408549000.0,\"EBITDA\":-334447000.0,\"EBIT\":-356969000.0,\"Net Interest Income\":16251000.0,\"Interest Expense\":4058000.0,\"Interest Income\":20309000.0,\"Normalized Income\":-373705000.0,\"Net Income From Continuing And Discontinued Operation\":-373705000.0,\"Total Expenses\":2067072000.0,\"Total Operating Income As Reported\":-161201000.0,\"Diluted Average Shares\":2063793000.0,\"Basic Average Shares\":2063793000.0,\"Diluted EPS\":-0.18,\"Basic EPS\":-0.18,\"Diluted NI Availto Com Stockholders\":-373705000.0,\"Average Dilution Earnings\":0.0,\"Net Income Common Stockholders\":-373705000.0,\"Otherunder Preferred Stock Dividend\":null,\"Net Income\":-373705000.0,\"Minority Interests\":-2611000.0,\"Net Income Including Noncontrolling Interests\":-371094000.0,\"Net Income Continuous Operations\":-371094000.0,\"Tax Provision\":10067000.0,\"Pretax Income\":-361027000.0,\"Other Income Expense\":-216077000.0,\"Other Non Operating Income Expenses\":-216077000.0,\"Gain On Sale Of Security\":null,\"Net Non Operating Interest Income Expense\":16251000.0,\"Interest Expense Non Operating\":4058000.0,\"Interest Income Non Operating\":20309000.0,\"Operating Income\":-161201000.0,\"Operating Expense\":1658523000.0,\"Research And Development\":359679000.0,\"Selling General And Administration\":1298844000.0,\"Selling And Marketing Expense\":702511000.0,\"General And Administrative Expense\":596333000.0,\"Other Gand A\":596333000.0,\"Gross Profit\":1497322000.0,\"Cost Of Revenue\":408549000.0,\"Total Revenue\":1905871000.0,\"Operating Revenue\":1905871000.0},\"1640908800000\":{\"Tax Effect Of Unusual Items\":0.0,\"Tax Rate For Calcs\":0.27,\"Normalized EBITDA\":-469957000.0,\"Total Unusual Items\":null,\"Total Unusual Items Excluding Goodwill\



[get_balance_sheet] {'ticker': 'PLTR'}

[Result (get_balance_sheet)]


"{\"1703980800000\":{\"Treasury Shares Number\":0.0,\"Ordinary Shares Number\":2200127714.0,\"Share Issued\":2200127714.0,\"Total Debt\":229392000.0,\"Tangible Book Value\":3475561000.0,\"Invested Capital\":3475561000.0,\"Working Capital\":3392600000.0,\"Net Tangible Assets\":3475561000.0,\"Capital Lease Obligations\":229392000.0,\"Common Stock Equity\":3475561000.0,\"Total Capitalization\":3475561000.0,\"Total Equity Gross Minority Interest\":3560965000.0,\"Minority Interest\":85404000.0,\"Stockholders Equity\":3475561000.0,\"Gains Losses Not Affecting Retained Earnings\":801000.0,\"Other Equity Adjustments\":801000.0,\"Treasury Stock\":null,\"Retained Earnings\":-5649613000.0,\"Additional Paid In Capital\":9122173000.0,\"Capital Stock\":2200000.0,\"Common Stock\":2200000.0,\"Preferred Stock\":null,\"Total Liabilities Net Minority Interest\":961460000.0,\"Total Non Current Liabilities Net Minority Interest\":215442000.0,\"Other Non Current Liabilities\":10702000.0,\"Preferred Securities Outside Stock Equity\":null,\"Non Current Deferred Liabilities\":29524000.0,\"Non Current Deferred Revenue\":29524000.0,\"Long Term Debt And Capital Lease Obligation\":175216000.0,\"Long Term Capital Lease Obligation\":175216000.0,\"Long Term Debt\":null,\"Current Liabilities\":746018000.0,\"Current Deferred Liabilities\":456729000.0,\"Current Deferred Revenue\":456729000.0,\"Current Debt And Capital Lease Obligation\":54176000.0,\"Current Capital Lease Obligation\":54176000.0,\"Payables And Accrued Expenses\":235113000.0,\"Current Accrued Expenses\":175734000.0,\"Payables\":59379000.0,\"Total Tax Payable\":47257000.0,\"Accounts Payable\":12122000.0,\"Total Assets\":4522425000.0,\"Total Non Current Assets\":383807000.0,\"Other Non Current Assets\":153186000.0,\"Net PPE\":230621000.0,\"Accumulated Depreciation\":-102158000.0,\"Gross PPE\":332779000.0,\"Leases\":83139000.0,\"Construction In Progress\":2099000.0,\"Other Properties\":182863000.0,\"Machinery Furniture Equipment\":64678000.0,\"Properties\":0.0,\"Current Assets\":4138618000.0,\"Other Current Assets\":99285000.0,\"Restricted Cash\":370000.0,\"Prepaid Assets\":null,\"Receivables\":364784000.0,\"Accounts Receivable\":364784000.0,\"Cash Cash Equivalents And Short Term Investments\":3674179000.0,\"Other Short Term Investments\":2843132000.0,\"Cash And Cash Equivalents\":831047000.0},\"1672444800000\":{\"Treasury Shares Number\":null,\"Ordinary Shares Number\":2099075000.0,\"Share Issued\":2099075000.0,\"Total Debt\":249404000.0,\"Tangible Book Value\":2565326000.0,\"Invested Capital\":2565326000.0,\"Working Capital\":2453636000.0,\"Net Tangible Assets\":2565326000.0,\"Capital Lease Obligations\":249404000.0,\"Common Stock Equity\":2565326000.0,\"Total Capitalization\":2565326000.0,\"Total Equity Gross Minority Interest\":2642437000.0,\"Minority Interest\":77111000.0,\"Stockholders Equity\":2565326000.0,\"Gains Losses Not Affecting Retained Earnings\":-5333000.0,\"Other Equity Adjustments\":-5333000.0,\"Treasury Stock\":null,\"Retained Earnings\":-5859438000.0,\"Additional Paid In Capital\":8427998000.0,\"Capital Stock\":2099000.0,\"Common Stock\":2099000.0,\"Preferred Stock\":null,\"Total Liabilities Net Minority Interest\":818802000.0,\"Total Non Current Liabilities Net Minority Interest\":230861000.0,\"Other Non Current Liabilities\":12655000.0,\"Preferred Securities Outside Stock Equity\":null,\"Non Current Deferred Liabilities\":13901000.0,\"Non Current Deferred Revenue\":13901000.0,\"Long Term Debt And Capital Lease Obligation\":204305000.0,\"Long Term Capital Lease Obligation\":204305000.0,\"Long Term Debt\":null,\"Current Liabilities\":587941000.0,\"Current Deferred Liabilities\":325339000.0,\"Current Deferred Revenue\":325339000.0,\"Current Debt And Capital Lease Obligation\":45099000.0,\"Current Capital Lease Obligation\":45099000.0,\"Payables And Accrued Expenses\":217503000.0,\"Current Accrued Expenses\":131389000.0,\"Payables\":86114000.0,\"Total Tax Payable\":41326000.0,\"Acc



[get_daily_stock_performance] {'ticker': 'PLTR'}

[Result (get_daily_stock_performance)]


"{\"Open\":{\"1727668800000\":36.5800018311,\"1727755200000\":37.2099990845,\"1727841600000\":36.2999992371,\"1727928000000\":37.4500007629,\"1728014400000\":39.6500015259,\"1728273600000\":39.9000015259,\"1728360000000\":39.2900009155,\"1728446400000\":41.5499992371,\"1728532800000\":42.9199981689,\"1728619200000\":43.4000015259,\"1728878400000\":43.7200012207,\"1728964800000\":43.4749984741,\"1729051200000\":42.4000015259,\"1729137600000\":42.1500015259,\"1729224000000\":42.0149993896,\"1729483200000\":42.8600006104,\"1729569600000\":42.6899986267,\"1729656000000\":42.8499984741,\"1729742400000\":42.8899993896,\"1729828800000\":43.8499984741,\"1730088000000\":45.0499992371,\"1730174400000\":44.9000015259,\"1730260800000\":44.5800018311,\"1730347200000\":42.7900009155,\"1730433600000\":41.9300003052,\"1730696400000\":41.0800018311,\"1730782800000\":47.8600006104,\"1730869200000\":53.4399986267,\"1730955600000\":55.8199996948,\"1731042000000\":55.5699996948,\"1731301200000\":60.1329994202,\"1731387600000\":59.0099983215,\"1731474000000\":61.5200004578,\"1731560400000\":61.0499992371,\"1731646800000\":61.0099983215,\"1731906000000\":64.9199981689,\"1731992400000\":59.5099983215,\"1732078800000\":62.5900001526,\"1732165200000\":62.0400009155,\"1732251600000\":61.5999984741,\"1732510800000\":66.9000015259,\"1732597200000\":64.5699996948,\"1732683600000\":66.2399978638,\"1732856400000\":65.8899993896,\"1733115600000\":67.4400024414,\"1733202000000\":66.4100036621,\"1733288400000\":71.1299972534,\"1733374800000\":70.1100006104,\"1733461200000\":72.9499969482,\"1733720400000\":80.5800018311,\"1733806800000\":71.9899978638,\"1733893200000\":72.9800033569,\"1733979600000\":72.3000030518,\"1734066000000\":73.4899978638,\"1734325200000\":75.0599975586,\"1734411600000\":73.2799987793,\"1734498000000\":74.3000030518,\"1734584400000\":73.9899978638,\"1734670800000\":72.6399993896,\"1734930000000\":80.6500015259,\"1735016400000\":81.0899963379,\"1735189200000\":83.1600036621,\"1735275600000\":81.4199981689,\"1735534800000\":77.0400009155},\"High\":{\"1727668800000\":37.3199996948,\"1727755200000\":37.5600013733,\"1727841600000\":37.6800003052,\"1727928000000\":39.2900009155,\"1728014400000\":40.2900009155,\"1728273600000\":40.1500015259,\"1728360000000\":41.4900016785,\"1728446400000\":43.3400001526,\"1728532800000\":43.8779983521,\"1728619200000\":44.3800010681,\"1728878400000\":44.3899993896,\"1728964800000\":43.6800003052,\"1729051200000\":42.4900016785,\"1729137600000\":42.7000007629,\"1729224000000\":42.9900016785,\"1729483200000\":43.4000015259,\"1729569600000\":43.4900016785,\"1729656000000\":43.6800003052,\"1729742400000\":43.7299995422,\"1729828800000\":45.0699996948,\"1730088000000\":45.1399993896,\"1730174400000\":45.0699996948,\"1730260800000\":44.7999992371,\"1730347200000\":43.3199996948,\"1730433600000\":42.5699996948,\"1730696400000\":42.0400009155,\"1730782800000\":51.5750007629,\"1730869200000\":55.6899986267,\"1730955600000\":56.4700012207,\"1731042000000\":58.4799995422,\"1731301200000\":62.0800018311,\"1731387600000\":60.0900001526,\"1731474000000\":63.3899993896,\"1731560400000\":61.1699981689,\"1731646800000\":66.0,\"1731906000000\":64.9199981689,\"1731992400000\":63.0099983215,\"1732078800000\":63.0800018311,\"1732165200000\":63.4000015259,\"1732251600000\":64.4400024414,\"1732510800000\":67.8799972534,\"1732597200000\":65.8499984741,\"1732683600000\":67.5599975586,\"1732856400000\":67.1600036621,\"1733115600000\":67.8799972534,\"1733202000000\":71.3700027466,\"1733288400000\":71.1800003052,\"1733374800000\":72.9800033569,\"1733461200000\":76.8199996948,\"1733720400000\":80.9100036621,\"1733806800000\":75.5599975586,\"1733893200000\":73.0,\"1733979600000\":75.3799972534,\"1734066000000\":76.0999984741,\"1734325200000\":77.8199996948,\"1734411600000\":75.3199996948,\"1734498000000\":77.5999984741,\"1734584400000\":76.5899963379,\"1734670800000\":80.6600036621,\"1734930000000\":82.0199966431,\"1735016400000\":84.796997


To evaluate whether Palantir Technologies Inc. (Ticker: PLTR) is a good investment, let's analyze its financial performance, balance sheet, and recent stock performance.

### Financial Performance (Income Statement)
- **Total Revenue**: $2.23 billion
- **Net Income**: $209.83 million
- **Diluted EPS**: $0.09
- **Total Expenses**: $2.11 billion
- **EBITDA**: $273.92 million

Palantir has shown a positive net income and revenue growth, which is a good sign. However, the diluted EPS is relatively low, indicating that while the company is profitable, the earnings per share are modest.

### Balance Sheet
- **Total Assets**: $4.52 billion
- **Total Liabilities**: $961.46 million
- **Total Equity**: $3.48 billion
- **Working Capital**: $3.39 billion
- **Total Debt**: $229.39 million

Palantir has a strong balance sheet with significantly more assets than liabilities, indicating financial stability. The working capital is also healthy, suggesting that the company can cover its short-term obli